#Automatic Unit Tests part. 1

In [ ]:
%%writefile item_crud_service.py
from flask import Flask, request, jsonify

app = Flask(__name__)

# Sample data store (replace with a database)
data = []

# Create
@app.route('/items', methods=['POST'])
def create_item():
  new_item = request.get_json()
  if not new_item:
    return jsonify({'error': 'Missing item data'}), 400

  data.append(new_item)
  return jsonify({'message': 'Item created successfully'}), 201

# Read All
@app.route('/items', methods=['GET'])
def get_all_items():
  return jsonify(data)

# Read One
@app.route('/items/<int:item_id>', methods=['GET'])
def get_item(item_id):
  for item in data:
    if item['id'] == item_id:
      return jsonify(item)
  return jsonify({'error': 'Item not found'}), 404

# Update
@app.route('/items/<int:item_id>', methods=['PUT'])
def update_item(item_id):
  update_data = request.get_json()
  if not update_data:
    return jsonify({'error': 'Missing update data'}), 400

  for i, item in enumerate(data):
    if item['id'] == item_id:
      data[i] = update_data
      return jsonify({'message': 'Item updated successfully'})
  return jsonify({'error': 'Item not found'}), 404

# Delete
@app.route('/items/<int:item_id>', methods=['DELETE'])
def delete_item(item_id):
  for i, item in enumerate(data):
    if item['id'] == item_id:
      del data[i]
      return jsonify({'message': 'Item deleted successfully'})
  return jsonify({'error': 'Item not found'}), 404

if __name__ == '__main__':
  app.run(debug=True)

Writing item_crud_service.py


In [ ]:
!pip install pylint

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.6/515.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.6/275.6 kB 28.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.5 MB/s eta 0:00:00


In [ ]:
!pylint item_crud_service.py

************* Module item_crud_service
item_crud_service.py:11:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
item_crud_service.py:12:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
item_crud_service.py:13:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
item_crud_service.py:15:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
item_crud_service.py:16:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
item_crud_service.py:21:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
item_crud_service.py:26:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)
item_crud_service.py:27:0: W0311: Bad indentation. Found 4 spaces, expected 8 (bad-indentation)
item_crud_service.py:28:0: W0311: Bad indentation. Found 6 spaces, expected 12 (bad-indentation)
item_crud_service.py:29:0: W0311: Bad indentation. Found 2 spaces, expected 4 (bad-indentation)


In [ ]:
%%writefile test_item_crud_service.py
import pytest
import json
from item_crud_service import app, data

@pytest.fixture
def client():
    app.testing = True
    with app.test_client() as client:
        yield client

@pytest.fixture
def sample_data():
    # Sample data for testing
    item1 = {'id': 1, 'name': 'Item A', 'description': 'First item'}
    item2 = {'id': 2, 'name': 'Item B', 'description': 'Second item'}
    data.extend([item1, item2])

    yield

    # Teardown - clear the data after each test
    data.clear()

def test_create_item(client, sample_data):
    new_item = {'id': 3, 'name': 'Item C', 'description': 'Third item'}
    response = client.post('/items', json=new_item)
    assert response.status_code == 201
    assert new_item in data

def test_get_all_items(client, sample_data):
    response = client.get('/items')
    assert response.status_code == 200
    assert json.loads(response.data) == data

def test_get_item(client, sample_data):
    response = client.get('/items/1')
    assert response.status_code == 200
    assert json.loads(response.data) == {'id': 1, 'name': 'Item A', 'description': 'First item'}

    response = client.get('/items/99')  # Item not found
    assert response.status_code == 404

def test_update_item(client, sample_data):
    updated_item = {'id': 1, 'name': 'Updated Item A', 'description': 'Updated description'}
    response = client.put('/items/1', json=updated_item)
    assert response.status_code == 200
    assert updated_item in data

    response = client.put('/items/99', json=updated_item)  # Item not found
    assert response.status_code == 404

def test_delete_item(client, sample_data):
    response = client.delete('/items/1')
    assert response.status_code == 200
    assert {'id': 1, 'name': 'Item A', 'description': 'First item'} not in data

    response = client.delete('/items/99')  # Item not found
    assert response.status_code == 404

Writing test_item_crud_service.py


In [ ]:
!pytest test_item_crud_service.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: anyio-3.7.1
collected 5 items                                                                                  

test_item_crud_service.py .....                                                              [100%]

======================================== 5 passed in 0.18s =========================================
